# Выполнено Зверьковой Елизаветой Алексеевной

# Сессия 2

## В данной сессии нам предстоит сделать следующее:
- выполнить предварительную обработку данных
- найти ключевые слова, биграммы/тиграммы для дальнейшего постороения модели

In [1]:
#импорты
import pandas as pd
import numpy as np
import re
import csv
import pymorphy2
from sklearn.feature_extraction.text import CountVectorizer
import nltk
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('df.csv', index_col=0) #сохраненный ранее датасет

In [3]:
df

,date,petition_text,reason_petition,category_petition
0,2020-11-13,"1й подъезд, 5 этаж. В коридоре над",Неудовлетворительное состояние парадной,Содержание МКД
1,2020-11-05,Маленькмй лифт изрисован внутри.,Неудовлетворительное состояние парадной,Содержание МКД
2,2020-11-03,реклама на светофоре,Несанкционированные надписи / объявления на те...,Благоустройство
3,2020-11-23,"1й подъезд, 5 этаж, квартирный холл. Рааботник...",Неудовлетворительное состояние парадной,Содержание МКД
4,2020-12-16,незаконная решетка на фасаде,Самовольное размещение наружн. блоков систем к...,Нарушение правил пользования общим имуществом
...,...,...,...,...
59884,2021-04-23,Систематическое блокирование парковки крупнога...,Мусор на внутридворовой территории,Благоустройство
59885,2021-04-23,"Ул. Маршала Захарова, д. 14/4.высокая точка за...",Неочищенные урны,Благоустройство
59886,2021-04-23,"люк выдавило, гранитное мощение вокруг люка вы...","Неудовлетворительное состояние колодца, люка",Повреждения или неисправность элементов улично...
59887,2021-04-23,Повреждён асфальт вокруг люка,"Неудовлетворительное состояние колодца, люка",Повреждения или неисправность элементов улично...


---

# 2.1. Предварительная обработка данных

## В данном этапе необходимо:
- выполнить разведочный анализ данных для составления плана предварительной обработки текста обращения граждан
- провести предварительную обработку данных с помощью методов NLP, а именно токенизацию, удаление стоп-слов, удаление пунктуации, удаление спецсимволов и лемматизацию

### Токенезация

Токенезация - это процесс разделения текста на отдельные токены, то есть небольшие части. В нашем случае нашими токенами будут являться слова, то есть мы проведем непосредственно токенезацию слов.

In [5]:
#импорт библиотеки и модуля
import nltk
from nltk.tokenize import word_tokenize

#токенезация
df['petition_text'] = df.apply(lambda row: nltk.word_tokenize(row['petition_text']), axis=1)

### Удаление символов, знаков пунктуации

С помощью регулярных выражений очистим текст от спецсимволов и пунктуации и приведем все к строчным буквам

In [6]:
df['petition_text'] = df['petition_text'].apply(lambda x: re.sub("[^a-zA-Zа-яА-Я]", " ", str(x).lower().strip()))

### Удаление стопслов

Стоп-слова — это слова, которые присутствуют в любом языке и не привносят никакой полезной информации в предложение. ИМенно поэтому на необходимо их удалить.

In [8]:
#стоп-слова русского языка
stopwords =  ["ул", "д", "согласно", "c","а","алло","без","белый","близко","более","больше","большой","будем","будет","будете","будешь","будто","буду","будут","будь","бы","бывает","бывь","был","была","были","было","быть","в","важная","важное","важные","важный","вам","вами","вас","ваш","ваша","ваше","ваши","вверх","вдали","вдруг","ведь","везде","вернуться","весь","вечер","взгляд","взять","вид","видел","видеть","вместе","вне","вниз","внизу","во","вода","война","вокруг","вон","вообще","вопрос","восемнадцатый","восемнадцать","восемь","восьмой","вот","впрочем","времени","время","все","все еще","всегда","всего","всем","всеми","всему","всех","всею","всю","всюду","вся","всё","второй","вы","выйти","г","где","главный","глаз","говорил","говорит","говорить","год","года","году","голова","голос","город","да","давать","давно","даже","далекий","далеко","дальше","даром","дать","два","двадцатый","двадцать","две","двенадцатый","двенадцать","дверь","двух","девятнадцатый","девятнадцать","девятый","девять","действительно","дел","делал","делать","делаю","дело","день","деньги","десятый","десять","для","до","довольно","долго","должен","должно","должный","дом","дорога","друг","другая","другие","других","друго","другое","другой","думать","душа","е","его","ее","ей","ему","если","есть","еще","ещё","ею","её","ж","ждать","же","жена","женщина","жизнь","жить","за","занят","занята","занято","заняты","затем","зато","зачем","здесь","земля", "здравствуйте","знать","значит","значить","й","и","иди","идти","из","или","им","имеет","имел","именно","иметь","ими","имя","иногда","их","к","каждая","каждое","каждые","каждый","кажется","казаться","как","какая","какой","кем","книга","когда","кого","ком","комната","кому","конец","конечно","которая","которого","которой","которые","который","которых","кроме","кругом","кто","куда","лежать","лет","ли","лицо","лишь","лучше","любить","люди","м","маленький","маленькмй", "мало","мать","машина","между","меля","менее","меньше","меня","место","миллионов","мимо","минута","мир","мира","мне","много","многочисленная","многочисленное","многочисленные","многочисленный","мной","мною","мог","могу","могут","мож","может","может быть","можно","можхо","мои","мой","мор","москва","мочь","моя","моё","мы","на","наверху","над","надо","назад","наиболее","найти","наконец","нам","нами","народ","нас","начала","начать","наш","наша","наше","наши","не","него","недавно","недалеко","нее","ней","некоторый","нельзя","нем","немного","нему","непрерывно","нередко","несколько","нет","нею","неё","ни","нибудь","ниже","низко","никакой","никогда","никто","никуда","ним","ними","них","ничего","ничто","но","новый","нога","ночь","ну","нужно","нужный","нх","о","об","оба","обычно","один","одиннадцатый","одиннадцать","однажды","однако","одного","одной","оказаться","окно","около","он","она","они","оно","опять","особенно","остаться","от","ответить","отец","откуда","отовсюду","отсюда","очень","первый","перед","писать","плечо","по","под","подойди","подумать","пожалуйста","позже","пойти","пока","пол","получить","помнить","понимать","понять","пор","пора","после","последний","посмотреть","посреди","потом","потому","почему","почти","правда","прекрасно","при","про","просто","против","процентов","путь","пятнадцатый","пятнадцать","пятый","пять","работа","работать","раз","разве","рано","раньше","ребенок","решить","россия","рука","русский","ряд","рядом","с","с кем","сам","сама","сами","самим","самими","самих","само","самого","самой","самом","самому","саму","самый","свет","свое","своего","своей","свои","своих","свой","свою","сделать","сеаой","себе","себя","сегодня","седьмой","сейчас","семнадцатый","семнадцать","семь","сидеть","сила","сих","сказал","сказала","сказать","сколько","слишком","слово","случай","смотреть","сначала","снова","со","собой","собою","советский","совсем","спасибо","спросить","сразу","стал","старый","стать","стол","сторона","стоять","страна","суть","считать","т","та","так","такая","также","таки","такие","такое","такой","там","твои","твой","твоя","твоё","те","тебе","тебя","тем","теми","теперь","тех","то","тобой","тобою","товарищ","тогда","того","тоже","только","том","тому","тот","тою","третий","три","тринадцатый","тринадцать","ту","туда","тут","ты","тысяч","у","увидеть","уж","уже","улица","уметь","утро","хороший","хорошо","хотел бы","хотеть","хоть","хотя","хочешь","час","часто","часть","чаще","чего","человек","чем","чему","через","четвертый","четыре","четырнадцатый","четырнадцать","что","чтоб","чтобы","чуть","шестнадцатый","шестнадцать","шестой","шесть","эта","эти","этим","этими","этих","это","этого","этой","этом","этому","этот","эту","я","являюсь"]

In [9]:
#удаление стоп-слов
df['petition_text'] = df['petition_text'].apply(lambda x: " ".join(word for word in x.split() if word not in stopwords))

### Лемматизация

Лемматизация - процесс сокращения слова до его основы, по-другому же леммы. Таким образом данный процесс приводит каждое слова в его нормальную (начальную) форму

In [10]:
#импорт
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [11]:
df["petition_text"] = df["petition_text"].apply(lambda x: " ".join(morph.parse(word)[0].normal_form for word in x.split()))

## Вывод:
На этапе предварительной обработки данных мы:
- разбили текста жалоб на отдельные токены, то бишь разделили на слова
- очистили текст от неиформативных слов (стоп-слов)
- удалили все спецсимволы и знаки пунктуации
- привели слова к их начальной форме

# 2.2. Поиск ключевых слов/биграмм/триграмм

## Сейчас необходимо выполнить:
- поиск ключевых слов/биграмм/триграмм в тексте обращения
- добавление найденных ключевых слов/биграмм/триграмм как новые признаки в набор данных

Для поиска ключевых слов/биграмм/триграмм мы будем испльзовать CountVectorizer. Он предоставляет простой способ как токенизировать коллекцию текстовых документов и создать словарь известных слов, так и кодировать новые документы, используя этот словарь и также позволяет найти ключевые слова/словосочетания из двух/трех слов

Для оптимизации процесса поиска ключевых слов создадим функцию `key_words`

In [15]:
def key_words(filename): 
    cv = CountVectorizer(binary=False,ngram_range=(0,3), max_features=5) 
    cv.fit_transform(filename)
    cv.vocabulary_
    words = str(cv.get_feature_names())
    return words

`ngram_range` как раз и отвечает за количество слов в словосочетании, указывая диапазон (0,3) нам будут выдаваться как и по одному слову, так и словосочетания, содержащие два и три слова

`max_fetures` необходимо для установления максимального количества ключевых слов/биграмм/триграмм в одной жалобе. В нашем случае выставила max_features=5, так как это наибольное оптимальное количество

In [16]:
#переменная, где будут находится найденные ключевые слова
KW= []

#поиск слов из каждой строки, то есть из каждой жалобы
for i in df['petition_text'].iloc[:]:
    try:
        kw = key_words([i])
        KW.append(kw)
    except Exception:
        continue

#добавление найденных слов в набор данных как новый атрибут
df['key_words'] = KW

При процессе поиска ключевых слов они были сохраненны со спецсимволами, поэтому их мы удаляем с помощью регулярных выражений

In [19]:
df['key_words'] = df['key_words'].apply(lambda x: re.sub(r'(n)|(\")|(\')|(\[)|(\])|(^\,)', '', str(x)).lower().strip())

## Вывод
На данном этапе мы:
- с помощью CountVectorizer мы нашли ключевые слова, биграммы и тиграммы для каждого текста обращения
- дополнили набор данных найденными словами

---

# Итог:
В этой сессии мы проделали следующую работу:
- предобработали текст, а именно очистили текст от спецсимволов, знаков пунктуации, стоп-слов и произвели процессы токенезация и лемматизации
- нашли ключевые слова, биграммы, тиграммы и добавили их в новую колонку под названием 'key_words'

In [25]:
df.to_csv('df.csv') #сохранение датасета